### Feature

* Convert data to the format that the model can consume.

#### Input
* train.parquet
* test.parquet
* encoder_num.rds

#### Output
* train_X.parquet
* train_Y.parquet
* test_X.parquet
* test_Y.parquet

In [1]:
suppressMessages(library(dplyr))

In [2]:
# file and directory info
project_dir = '/Users/chou/Desktop/mlflow_dvc_cookiecutter/DSProjectTemplate'
interim_folder = "/data/interim/"
processed_folder = "/data/processed/"

# input
train = arrow::read_parquet(paste0(project_dir, interim_folder, "train.parquet"))
test = arrow::read_parquet(paste0(project_dir, interim_folder, "test.parquet"))
encoder_num = readRDS(paste0(project_dir, processed_folder, "encoder_num.rds"))

# functions 
Featurize_train = function(train, encoder_num){
    # numeric
    var_num = names(encoder_num)
    train_var_impute = purrr::map2(train[, var_num], encoder_num, function(.x, .y) ifelse(is.na(.x), .y$impute, .x)) %>% as_tibble(.)
    train_var_scale = purrr::map2(train_var_impute[, var_num], encoder_num, function(.x, .y) (.x - .y$mean)/.y$std ) %>% as_tibble(.)
                                
    X = train_var_scale
    Y = train %>% select(quality)
                               
    return(list(X = X, Y = Y))
}

Featurize_test = function(test, encoder_num){
    # numeric
    var_num = names(encoder_num)
    test_var_impute = purrr::map2(test[, var_num], encoder_num, function(.x, .y) ifelse(is.na(.x), .y[[1]], .x)) %>% as_tibble(.)
    test_var_scale = purrr::map2(test_var_impute[, var_num], encoder_num, function(.x, .y) (.x - .y$mean)/.y$std ) %>% as_tibble(.)
                                  
    X = test_var_scale 
    Y = test %>% select(quality)
                              
    return(list(X = X, Y = Y))
}

# output and save
feature_train = Featurize_train(train, encoder_num)
feature_test = Featurize_test(test, encoder_num)
                                  
arrow::write_parquet(feature_train$X, paste0(project_dir, interim_folder, "train_X.parquet"))
arrow::write_parquet(feature_train$Y, paste0(project_dir, interim_folder, "train_Y.parquet"))
arrow::write_parquet(feature_test$X, paste0(project_dir, interim_folder, "test_X.parquet"))
arrow::write_parquet(feature_test$Y, paste0(project_dir, interim_folder, "test_Y.parquet"))

In [3]:
lapply(feature_train, head)
lapply(feature_test, head)

$X
# A tibble: 6 x 11
  `fixed acidity` `volatile acidi… `citric acid` `residual sugar` chlorides
            <dbl>            <dbl>         <dbl>            <dbl>     <dbl>
1           1.74           -0.285        -0.0322           -0.756    0.332 
2          -0.770           0.0109        0.955             0.235   -0.0316
3           1.02            0.0109        2.03              1.15     0.0139
4          -1.13            0.405         0.462            -0.598    3.11  
5          -1.37           -0.186        -0.773             2.28     0.605 
6          -0.173          -0.974        -0.444             0.770   -0.304 
# … with 6 more variables: `free sulfur dioxide` <dbl>, `total sulfur
#   dioxide` <dbl>, density <dbl>, pH <dbl>, sulphates <dbl>, alcohol <dbl>

$Y
# A tibble: 6 x 1
  quality
    <int>
1       5
2       6
3       5
4       6
5       5
6       7

$X
# A tibble: 6 x 11
  `fixed acidity` `volatile acidi… `citric acid` `residual sugar` chlorides
            <dbl>            <dbl>         <dbl>            <dbl>     <dbl>
1          1.50             0.0109         0.544            0.116    0.196 
2          0.186           -0.0877         0.215            2.85    -0.0316
3         -0.0533          -0.186          0.708           -0.915    0.150 
4          0.902            3.85          -1.60            -0.954    1.29  
5          0.186            0.504         -0.114           -1.01     0.332 
6         -0.770            1.78           0               -0.380    0.923 
# … with 6 more variables: `free sulfur dioxide` <dbl>, `total sulfur
#   dioxide` <dbl>, density <dbl>, pH <dbl>, sulphates <dbl>, alcohol <dbl>

$Y
# A tibble: 6 x 1
  quality
    <int>
1       6
2       6
3       8
4       5
5       6
6       5

In [4]:
lapply(feature_train, dim)
lapply(feature_test, dim)

$X
[1] 3918   11

$Y
[1] 3918    1

$X
[1] 980  11

$Y
[1] 980   1